In [130]:
#import libaries
import pandas as pd
import numpy as np 
import plotly.express as px 
from sklearn.impute import KNNImputer ,SimpleImputer
from datasist.structdata import detect_outliers


In [131]:
phalstine_df=pd.read_csv('../data/raw/fatalities_isr_pse_conflict_2000_to_2023.csv')
df=phalstine_df.copy()
df

,name,date_of_event,age,citizenship,event_location,event_location_district,event_location_region,date_of_death,gender,took_part_in_the_hostilities,place_of_residence,place_of_residence_district,type_of_injury,ammunition,killed_by,notes
0,'Abd a-Rahman Suleiman Muhammad Abu Daghash,9/24/2023,32.0,Palestinian,Nur Shams R.C.,Tulkarm,West Bank,9/24/2023,M,NaN,Nur Shams R.C.,Tulkarm,gunfire,live ammunition,Israeli security forces,Fatally shot by Israeli forces while standing ...
1,Usayed Farhan Muhammad 'Ali Abu 'Ali,9/24/2023,21.0,Palestinian,Nur Shams R.C.,Tulkarm,West Bank,9/24/2023,M,NaN,Nur Shams R.C.,Tulkarm,gunfire,live ammunition,Israeli security forces,Fatally shot by Israeli forces while trying to...
2,'Abdallah 'Imad Sa'ed Abu Hassan,9/22/2023,16.0,Palestinian,Kfar Dan,Jenin,West Bank,9/22/2023,M,NaN,al-Yamun,Jenin,gunfire,live ammunition,Israeli security forces,Fatally shot by soldiers while firing at them ...
3,Durgham Muhammad Yihya al-Akhras,9/20/2023,19.0,Palestinian,'Aqbat Jaber R.C.,Jericho,West Bank,9/20/2023,M,NaN,'Aqbat Jaber R.C.,Jericho,gunfire,live ammunition,Israeli security forces,Shot in the head by Israeli forces while throw...
4,Raafat 'Omar Ahmad Khamaisah,9/19/2023,15.0,Palestinian,Jenin R.C.,Jenin,West Bank,9/19/2023,M,NaN,Jenin,Jenin,gunfire,live ammunition,Israeli security forces,Wounded by soldiers’ gunfire after running awa...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11119,Binyamin Herling,10/19/2000,64.0,Israeli,Nablus,Nablus,West Bank,10/19/2000,M,Israelis,Kedumim,Tulkarm,gunfire,live ammunition,Palestinian civilians,Killed while hiking on Mt. Eival.
11120,Farid Musa 'Issa a-Nesasreh,10/17/2000,28.0,Palestinian,Beit Furik,Nablus,West Bank,10/17/2000,M,Unknown,Beit Furik,Nablus,gunfire,NaN,Israeli civilians,Killed by a settler from Itamar while harvesti...
11121,Hillel Lieberman,10/7/2000,36.0,Israeli,Nablus,Nablus,West Bank,10/7/2000,M,Israelis,Elon Moreh,Nablus,gunfire,live ammunition,Palestinian civilians,His body was found a day after he disappeared.
11122,Fahed Mustafa 'Odeh Baker,10/7/2000,21.0,Palestinian,Bidya,Salfit,West Bank,10/7/2000,M,No,Bidya,Salfit,gunfire,NaN,Israeli civilians,Killed by settlers who rioted in Biddya village.


In [132]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11124 entries, 0 to 11123
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   name                          11124 non-null  object 
 1   date_of_event                 11124 non-null  object 
 2   age                           10995 non-null  float64
 3   citizenship                   11124 non-null  object 
 4   event_location                11124 non-null  object 
 5   event_location_district       11124 non-null  object 
 6   event_location_region         11124 non-null  object 
 7   date_of_death                 11124 non-null  object 
 8   gender                        11104 non-null  object 
 9   took_part_in_the_hostilities  9694 non-null   object 
 10  place_of_residence            11056 non-null  object 
 11  place_of_residence_district   11056 non-null  object 
 12  type_of_injury                10833 non-null  object 
 13  a

In [133]:
df.isnull().sum()

name                               0
date_of_event                      0
age                              129
citizenship                        0
event_location                     0
event_location_district            0
event_location_region              0
date_of_death                      0
gender                            20
took_part_in_the_hostilities    1430
place_of_residence                68
place_of_residence_district       68
type_of_injury                   291
ammunition                      5253
killed_by                          0
notes                            280
dtype: int64

In [134]:
df.isnull().sum()/len(df)*100

name                             0.000000
date_of_event                    0.000000
age                              1.159655
citizenship                      0.000000
event_location                   0.000000
event_location_district          0.000000
event_location_region            0.000000
date_of_death                    0.000000
gender                           0.179791
took_part_in_the_hostilities    12.855088
place_of_residence               0.611291
place_of_residence_district      0.611291
type_of_injury                   2.615965
ammunition                      47.222222
killed_by                        0.000000
notes                            2.517080
dtype: float64

In [135]:
df=df.drop(columns='ammunition',axis=1)

In [136]:
df['date_of_death']=pd.to_datetime(df['date_of_death'])
df['date_of_event']=pd.to_datetime(df['date_of_event'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11124 entries, 0 to 11123
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   name                          11124 non-null  object        
 1   date_of_event                 11124 non-null  datetime64[ns]
 2   age                           10995 non-null  float64       
 3   citizenship                   11124 non-null  object        
 4   event_location                11124 non-null  object        
 5   event_location_district       11124 non-null  object        
 6   event_location_region         11124 non-null  object        
 7   date_of_death                 11124 non-null  datetime64[ns]
 8   gender                        11104 non-null  object        
 9   took_part_in_the_hostilities  9694 non-null   object        
 10  place_of_residence            11056 non-null  object        
 11  place_of_residence_district 

In [137]:
df['year']=df['date_of_death'].dt.year
df['month']=df['date_of_death'].dt.month
df['month_name']=df['date_of_death'].dt.month_name()

In [138]:
df

,name,date_of_event,age,citizenship,event_location,event_location_district,event_location_region,date_of_death,gender,took_part_in_the_hostilities,place_of_residence,place_of_residence_district,type_of_injury,killed_by,notes,year,month,month_name
0,'Abd a-Rahman Suleiman Muhammad Abu Daghash,2023-09-24,32.0,Palestinian,Nur Shams R.C.,Tulkarm,West Bank,2023-09-24,M,NaN,Nur Shams R.C.,Tulkarm,gunfire,Israeli security forces,Fatally shot by Israeli forces while standing ...,2023,9,September
1,Usayed Farhan Muhammad 'Ali Abu 'Ali,2023-09-24,21.0,Palestinian,Nur Shams R.C.,Tulkarm,West Bank,2023-09-24,M,NaN,Nur Shams R.C.,Tulkarm,gunfire,Israeli security forces,Fatally shot by Israeli forces while trying to...,2023,9,September
2,'Abdallah 'Imad Sa'ed Abu Hassan,2023-09-22,16.0,Palestinian,Kfar Dan,Jenin,West Bank,2023-09-22,M,NaN,al-Yamun,Jenin,gunfire,Israeli security forces,Fatally shot by soldiers while firing at them ...,2023,9,September
3,Durgham Muhammad Yihya al-Akhras,2023-09-20,19.0,Palestinian,'Aqbat Jaber R.C.,Jericho,West Bank,2023-09-20,M,NaN,'Aqbat Jaber R.C.,Jericho,gunfire,Israeli security forces,Shot in the head by Israeli forces while throw...,2023,9,September
4,Raafat 'Omar Ahmad Khamaisah,2023-09-19,15.0,Palestinian,Jenin R.C.,Jenin,West Bank,2023-09-19,M,NaN,Jenin,Jenin,gunfire,Israeli security forces,Wounded by soldiers’ gunfire after running awa...,2023,9,September
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11119,Binyamin Herling,2000-10-19,64.0,Israeli,Nablus,Nablus,West Bank,2000-10-19,M,Israelis,Kedumim,Tulkarm,gunfire,Palestinian civilians,Killed while hiking on Mt. Eival.,2000,10,October
11120,Farid Musa 'Issa a-Nesasreh,2000-10-17,28.0,Palestinian,Beit Furik,Nablus,West Bank,2000-10-17,M,Unknown,Beit Furik,Nablus,gunfire,Israeli civilians,Killed by a settler from Itamar while harvesti...,2000,10,October
11121,Hillel Lieberman,2000-10-07,36.0,Israeli,Nablus,Nablus,West Bank,2000-10-07,M,Israelis,Elon Moreh,Nablus,gunfire,Palestinian civilians,His body was found a day after he disappeared.,2000,10,October
11122,Fahed Mustafa 'Odeh Baker,2000-10-07,21.0,Palestinian,Bidya,Salfit,West Bank,2000-10-07,M,No,Bidya,Salfit,gunfire,Israeli civilians,Killed by settlers who rioted in Biddya village.,2000,10,October


In [139]:
df.isnull().sum()/len(df)*100

name                             0.000000
date_of_event                    0.000000
age                              1.159655
citizenship                      0.000000
event_location                   0.000000
event_location_district          0.000000
event_location_region            0.000000
date_of_death                    0.000000
gender                           0.179791
took_part_in_the_hostilities    12.855088
place_of_residence               0.611291
place_of_residence_district      0.611291
type_of_injury                   2.615965
killed_by                        0.000000
notes                            2.517080
year                             0.000000
month                            0.000000
month_name                       0.000000
dtype: float64

In [140]:
col=['took_part_in_the_hostilities','notes','type_of_injury']
for i in col :
    df[i]=df[i].fillna('Unknown')
df.isnull().sum()/len(df)*100

name                            0.000000
date_of_event                   0.000000
age                             1.159655
citizenship                     0.000000
event_location                  0.000000
event_location_district         0.000000
event_location_region           0.000000
date_of_death                   0.000000
gender                          0.179791
took_part_in_the_hostilities    0.000000
place_of_residence              0.611291
place_of_residence_district     0.611291
type_of_injury                  0.000000
killed_by                       0.000000
notes                           0.000000
year                            0.000000
month                           0.000000
month_name                      0.000000
dtype: float64

In [141]:
df.drop_duplicates(subset=['name'],inplace=True), df.dropna(inplace=True)

(None, None)

In [142]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10933 entries, 0 to 11123
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   name                          10933 non-null  object        
 1   date_of_event                 10933 non-null  datetime64[ns]
 2   age                           10933 non-null  float64       
 3   citizenship                   10933 non-null  object        
 4   event_location                10933 non-null  object        
 5   event_location_district       10933 non-null  object        
 6   event_location_region         10933 non-null  object        
 7   date_of_death                 10933 non-null  datetime64[ns]
 8   gender                        10933 non-null  object        
 9   took_part_in_the_hostilities  10933 non-null  object        
 10  place_of_residence            10933 non-null  object        
 11  place_of_residence_district   109

In [143]:
df['age'] = df['age'].astype(int)

In [144]:
df.isnull().sum()

name                            0
date_of_event                   0
age                             0
citizenship                     0
event_location                  0
event_location_district         0
event_location_region           0
date_of_death                   0
gender                          0
took_part_in_the_hostilities    0
place_of_residence              0
place_of_residence_district     0
type_of_injury                  0
killed_by                       0
notes                           0
year                            0
month                           0
month_name                      0
dtype: int64

In [145]:
(df.name.value_counts()>1).value_counts()

count
False    10933
Name: count, dtype: int64

In [146]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10933 entries, 0 to 11123
Data columns (total 18 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   name                          10933 non-null  object        
 1   date_of_event                 10933 non-null  datetime64[ns]
 2   age                           10933 non-null  int32         
 3   citizenship                   10933 non-null  object        
 4   event_location                10933 non-null  object        
 5   event_location_district       10933 non-null  object        
 6   event_location_region         10933 non-null  object        
 7   date_of_death                 10933 non-null  datetime64[ns]
 8   gender                        10933 non-null  object        
 9   took_part_in_the_hostilities  10933 non-null  object        
 10  place_of_residence            10933 non-null  object        
 11  place_of_residence_district   109

In [154]:
df['name'].value_counts()>1

name
'Abd a-Rahman Suleiman Muhammad Abu Daghash    False
Khaled Yusef Ahmad a-Z'anin                    False
Shawqi 'Ali 'Omar a-Seikali                    False
'Ali 'Ali Hussein al-'Umari                    False
Muhammad Subhi Naji Barghish                   False
                                               ...  
Hussein 'Abd a-Naser Hussein al-Astal          False
Zakaria 'Ahed Subhi Baker                      False
'Abd a-Rahman Ibrahim Khalil a-Sarhi           False
Isma'il Muhammad Subhi Baker                   False
Wichlav Zalsevsky                              False
Name: count, Length: 10933, dtype: bool

In [155]:
len(df)

10933

To Save In Processed Folder

In [159]:
df.to_csv('../data/processed/datset_cleaned.csv')